In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-01"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
14076,2024-03-01,Eua Nba,21:30,Toronto Raptors,Golden State Warriors,2.21,1.71,237.5,1.86,1.94,1.5,1.99,1.60,noAU1erB,0.452489,0.584795,0.537634,0.515464,0.037284,0.4,0.2,NaN,NaN,252.154,86.063867,0.341315,208.394,62.334911,0.299120,252.154,280.166,276.25,264.60,0.0,0.0,0.0,0.0,0.180384,0.029773,0.153633,-3.22,-0.644,-1.878882,0.484210,0.2,-0.284210,6.00,1.200,0.591667,0.487362,0.7,0.212638
14077,2024-03-01,Eua Nba,22:00,Memphis Grizzlies,Portland Trail Blazers,1.56,2.53,208.5,1.81,2.07,-4.5,2.03,3.17,Is4h3q8e,0.641026,0.395257,0.552486,0.483092,0.036283,0.4,0.4,NaN,NaN,324.322,185.904486,0.573210,383.156,168.941039,0.440920,353.126,609.322,165.98,626.24,0.0,1.0,0.0,0.0,0.335400,0.094767,0.310039,3.04,0.608,0.921053,0.322980,0.4,0.077020,0.11,0.022,69.545455,0.207313,0.2,-0.007313
14078,2024-03-01,Eua Nba,22:00,New Orleans Pelicans,Indiana Pacers,1.51,2.68,236.5,1.85,2.07,-4.5,1.97,3.29,Q7701Nx8,0.662252,0.373134,0.540541,0.483092,0.035386,0.6,0.4,NaN,NaN,148.026,27.572003,0.186265,180.294,49.653088,0.275401,167.580,231.816,162.45,198.12,0.0,0.0,0.0,0.0,0.394900,0.079369,0.354898,-1.30,-0.260,-1.961538,0.694127,0.6,-0.094127,-2.22,-0.444,-3.783784,0.485709,0.4,-0.085709
14079,2024-03-01,Europa Euroliga,14:45,Fenerbahce,Baskonia,1.32,3.57,167.5,1.91,1.91,-8.5,2.00,7.50,O6DCeQPT,0.757576,0.280112,0.523560,0.523560,0.037688,0.4,0.8,NaN,NaN,136.386,41.424158,0.303727,166.478,65.350841,0.392549,118.354,245.494,103.50,225.63,0.0,0.0,0.0,0.0,0.650712,0.000000,0.818755,1.44,0.288,1.111111,0.794604,0.9,0.105396,-0.18,-0.036,-71.388889,0.377092,0.5,0.122908
14080,2024-03-01,Europa Euroliga,15:00,Zalgiris Kaunas,Olympiakos,2.14,1.76,154.5,1.91,1.91,1.5,2.01,1.57,QFavipYp,0.467290,0.568182,0.523560,0.523560,0.035472,0.6,0.6,NaN,NaN,225.728,49.372919,0.218727,225.860,206.649088,0.914943,179.422,239.898,174.40,148.20,0.0,0.0,0.0,0.0,0.137795,0.000000,0.173814,2.72,0.544,2.095588,0.000000,0.0,0.000000,-1.71,-0.342,-2.222222,0.471768,0.4,-0.071768
14081,2024-03-01,Europa Euroliga,16:00,Lyon-Villeurbanne,Milano,2.88,1.45,155.5,1.83,1.95,4.5,2.00,1.20,SKwNFTAo,0.347222,0.689655,0.546448,0.512821,0.036877,0.6,0.6,NaN,NaN,373.528,223.614957,0.598656,187.344,26.654592,0.142276,187.892,240.860,221.19,200.02,0.0,0.0,0.0,0.0,0.467050,0.044896,0.353553,-0.97,-0.194,-9.690722,0.482164,0.2,-0.282164,-0.86,-0.172,-2.616279,0.394197,0.2,-0.194197
14082,2024-03-01,Europa Euroliga,16:30,Barcelona,Monaco,1.61,2.42,161.5,1.83,1.91,-4.5,2.00,3.40,zosVD7ub,0.621118,0.413223,0.546448,0.523560,0.034341,0.8,0.2,NaN,NaN,125.978,21.913209,0.173945,154.938,38.854888,0.250777,115.834,155.756,99.51,145.80,0.0,0.0,0.0,0.0,0.284246,0.030251,0.366648,1.56,0.312,1.955128,0.755167,0.9,0.144833,-0.92,-0.184,-7.717391,0.513895,0.5,-0.013895
14083,2024-03-01,Europa Euroliga,16:30,Bayern,Crvena zvezda,1.54,2.73,160.5,1.85,2.09,-4.5,1.97,3.75,tOsREmQi,0.649351,0.366300,0.540541,0.478469,0.015651,0.8,0.6,NaN,NaN,152.938,75.865961,0.496057,120.414,26.305165,0.218456,129.724,130.138,110.36,89.90,0.0,0.0,0.0,0.0,0.394125,0.086145,0.440087,0.16,0.032,16.875000,0.726416,0.6,-0.126416,-2.65,-0.530,-3.264151,0.592852,0.5,-0.092852
14084,2024-03-01,Alemanha Pro A,15:30,Bochum,Bayreuth,2.02,1.72,171.5,1.85,1.85,1.5,1.87,1.59,hOMrL1wB,0.495050,0.581395,0.540541,0.540541,0.076445,0.2,0.8,NaN,NaN,180.100,47.596921,0.264281,132.700,24.074061,0.181417,169.834,117.054,100.87,121.66,0.0,0.0,0.0,0.0,0.113440,0.000000,0.11444

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,22:00,Eua Nba,Memphis Grizzlies,Portland Trail Blazers,208.5,1.81,1.0000,Over
1,22:00,Eua Nba,New Orleans Pelicans,Indiana Pacers,236.5,1.85,1.0000,Over
2,14:45,Europa Euroliga,Fenerbahce,Baskonia,167.5,1.91,1.0000,Over
3,16:30,Europa Euroliga,Barcelona,Monaco,161.5,1.83,1.0000,Over
4,16:30,Europa Euroliga,Bayern,Crvena zvezda,160.5,1.85,1.0000,Over
5,00:00,Eua Ncaa,California Baptist,Stephen F. Austin,134.5,1.80,0.9940,Over
6,00:30,Eua Ncaa,Washington,UCLA,139.5,1.80,1.0000,Over
7,00:30,Eua Ncaa,Washington State,USC,138.5,1.80,1.0000,Over
8,21:00,Eua Ncaa,Canisius,Mt. St. Marys,143.5,1.91,1.0000,Over
9,21:00,Eua Ncaa,Georgia Southern,Old Dominion,150.5,1.80,1.0000,Over
